In [ ]:
import tkinter as tk
import cv2
import numpy as np
import time

def calculate_speed(initial_pos, final_pos, time_diff, pixels_per_meter):
    if time_diff == 0:
        return 0
    distance = abs(final_pos - initial_pos) / pixels_per_meter
    speed_meters_per_second = distance / time_diff
    speed_kilometers_per_hour = speed_meters_per_second * 3.6
    return speed_kilometers_per_hour

def moving_average(new_value, old_value, alpha=0.8):
    return old_value * alpha + new_value * (1 - alpha)


terminate_flag = False
root = None

def start_object_speed_detection():
    global terminate_flag

    field_of_view_feet = 5.83     # total distance the object has to cover while being in the camera frame
    object_distance_feet = 3.25   # distance of object from camera
    field_of_view_meter = field_of_view_feet * 0.3048
    object_distance_meter = object_distance_feet * 0.3048

    cap = cv2.VideoCapture(0)
    prev_frame_time, initial_object_pos = 0, None
    bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
    smoothed_speed_kmh = 0

    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    _, frame = cap.read()
    height, width, _ = frame.shape
    pixels_per_meter = width / field_of_view_meter

    while not terminate_flag:
        ret, frame = cap.read()
        if not ret:
            break

        blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)
        fg_mask = bg_subtractor.apply(blurred_frame)
        _, fg_mask = cv2.threshold(fg_mask, 128, 255, cv2.THRESH_BINARY)

        kernel = np.ones((5, 5), np.uint8)
        fg_mask = cv2.erode(fg_mask, kernel, iterations=1)
        fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)

        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            moments = cv2.moments(largest_contour)
            object_position = int(moments["m10"] / moments["m00"]) if moments["m00"] != 0 else 0

            new_frame_time = time.time()
            time_diff = new_frame_time - prev_frame_time if prev_frame_time else 0

            if initial_object_pos is not None:
                speed_kmh = calculate_speed(initial_object_pos, object_position, time_diff, pixels_per_meter)
                smoothed_speed_kmh = moving_average(speed_kmh, smoothed_speed_kmh)

            initial_object_pos = object_position
            prev_frame_time = new_frame_time

            speed_text = f'Speed: {smoothed_speed_kmh:.2f} km/h'
            cv2.putText(frame, speed_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            x, y, w, h = cv2.boundingRect(largest_contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        else:
            speed_text = f'Last Speed: {smoothed_speed_kmh:.2f} km/h'
            cv2.putText(frame, speed_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow('Object Speed Detection', frame)
        root.update()
        time.sleep(0.02)

    cap.release()
    cv2.destroyAllWindows()

def quit_application():
    global terminate_flag
    terminate_flag = True

def setup_gui():
    global root
    root = tk.Tk()
    root.title("Speed Detection Control")

    start_button = tk.Button(root, text="Start Detection", command=start_object_speed_detection)
    start_button.pack(pady=20)

    quit_button = tk.Button(root, text="Quit", command=quit_application)
    quit_button.pack(pady=20)

setup_gui()
root.mainloop()

TclError: ignored